# Regression

Decision Tree는 regression에도 사용할 수 있다. 사이킷런의 `DecisionTreeRegressor`를 사용해 noisy quadratic dataset에 대해 `max_depth=2`로 학습을 시켜보자.

In [1]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

# Quadratic training set + noise
np.random.seed(42)
m = 200
X = np.random.rand(m, 1)
y = 4 * (X - 0.5) ** 2
y = y + np.random.randn(m, 1) / 10

tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=2,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [2]:
from sklearn.tree import export_graphviz

export_graphviz(
        tree_reg, 
        out_file="./noisy_quadratic_tree.dot",
        feature_names=["x1"],
        rounded=True,
        filled=True
    )

In [3]:
!dot -Tpng ./noisy_quadratic_tree.dot -o ./images/noisy_quadratic_tree.png

<br/>
<img src="./images/noisy_quadratic_tree.png" alt="noisy_quadratic_tree" width="80%">
<br/>

Regression tree는 value를 prediction한다. 예를 들어, $x_1=0.6$인 sample의 경우, root node에서부터 tree를 순회하여 `value=0.111`인 leaf node에 도달하고 해당 값인 0.111을 prediction result로 반환한다.(아래 예제 코드 참고)

In [4]:
print(tree_reg.predict([[0.6]]))

[0.11063973]


prediction의 결과인 0.111은 해당 leaf node에 있는 110개의 training instance들의 average target value이다.

다음의 그림은 `max_depth`에 다른 두 regression model의 prediction을 나타낸 것이다. 왼쪽의 그림은 앞서 학습시킨 모델이며, 오른쪽은 `max_depth=3`을 설정한 모델이다.

<br/>
<img src="./images/two_decision_tree_regression_models.png" alt="two_decision_tree_regression_models">
<br/>

각 영역의 prediction은 해당 영역에 있는 target 값들의 평균이 된다. 알고리즘은 prediction값과 가능한 많은 instance가 가까이 위치하도록 영역을 분할한다.

Regression에서의 CART 알고리즘은 gini impurity 대신 MSE를 minimize 하도록 분할하는 것을 제외하고는 classification과 거의 비슷하게 동작한다. 다음은 알고리즘이 minimize하는 cost function이다.

**CART cost function for regression**

$J(k, t_k) = \dfrac{m_{\text{left}}}{m}\text{MSE}_{\text{left}} + \dfrac{m_{\text{right}}}{m}\text{MSE}_{\text{right}}$

$\text{where}\begin{equation} \begin{cases}\text{MSE}_{\text{node}} = \sum_{i \in \text{ node}} \big( \hat{y}_{\text{node}}-y^{(i)} \big)^2 \\ \hat{y}_{\text{node}} = \dfrac{1}{m_{\text{node}}} \sum_{i \in \text{ node}} y^{(i)} \end{cases} \end{equation}$

classification처럼 regression에서도 decision tree는 overfitting되기 쉬우며 이를 방지하기 위해 regularization을 사용할 수 있다. 아래 그림은 regularization을 적용하지 않은 모델과 `min_samples_leaf=10`을 적용한 모델을 비교한 것이다.

<br/>
<img src="./images/regression_tree_regularization.png" alt="regression_tree_regularization">
<br/>